## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense
import pandas as pd
import numpy as np

# Read the charity_data.csv from the provided cloud URL
app_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
app_df.head()

2025-04-28 17:03:07.668878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df.drop(columns = ['EIN', 'NAME'], inplace = True)

In [ ]:
# Determine the number of unique values in each column.
for field, count in app_df.nunique().items():
    print(field, count)
# app_df.nunique()

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_val_cnts = app_df['APPLICATION_TYPE'].value_counts()
app_type_val_cnts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_types_to_repl = [i for i, r in app_type_val_cnts.items() if r < 200]

# # Replace in dataframe
for app in app_types_to_repl:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
app_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [ ]:
app_df['CLASSIFICATION']

,CLASSIFICATION
0,C1000
1,C2000
2,C3000
3,C2000
4,C1000
...,...
34294,C1000
34295,C3000
34296,C2000
34297,C3000


In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = app_df['CLASSIFICATION'].value_counts()
classification_counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts.describe()

,count
count,71.000000
mean,483.084507
std,2242.207852
min,1.000000
25%,1.000000
50%,5.000000
75%,32.000000
max,17326.000000


In [ ]:
len(classification_counts)

71

In [ ]:
len(classification_counts.loc[classification_counts < 50])

56

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [i for i, r in classification_counts.items() if r < 100]

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls, 'Other')

# Check to make sure replacement was successful
app_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
Other,669
C1700,287
C4000,194


In [ ]:
app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_df = app_df.select_dtypes(include = ['object'])
categories = list(categorical_df.columns)

app_df = pd.concat([app_df, pd.get_dummies(categorical_df)], axis = 1)
app_df.drop(columns = categories, inplace = True)

In [ ]:
# Split our preprocessed data into our features and target arrays
target = 'IS_SUCCESSFUL'
y = app_df[target]
X = app_df.drop(columns = [target])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = \
  train_test_split(X, y, random_state = 42, stratify = y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
def io_dimensions(
    data: pd.DataFrame | pd.Series | np.ndarray
    ) -> int:
    if isinstance(data, pd.DataFrame) or (isinstance(data, np.ndarray) and len(data.shape) <= 2):
      dims = data.shape[1]
    elif isinstance(data, pd.Series):
      dims = 1
    else:
      raise TypeError('Bad argument type. Currently only excepts pandas and numpy objects.')
    return dims

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dims = io_dimensions(X_train_scaled)
output_dims = io_dimensions(y_train)

nn_model = Sequential(
    [
        InputLayer(shape = (input_dims,))                   # Input layer
        ,Dense(units = 18, activation = 'relu')             # 1st hidden layer
        ,Dense(units = 6, activation = 'relu')             # 2nd hidden layer
        ,Dense(units = output_dims, activation = 'sigmoid') # Output layer
    ]
)

# Check the structure of the model
nn_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_17 (Dense)                │ (None, 18)             │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 6)              │           114 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,021 (3.99 KB)

 Trainable params: 1,021 (3.99 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
nn_model.compile(
    loss = 'binary_crossentropy'
    ,optimizer = 'adam'
    ,metrics = ['accuracy']
)

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6817 - loss: 0.6169
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7293 - loss: 0.5582
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7310 - loss: 0.5563
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7346 - loss: 0.5494
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7326 - loss: 0.5495
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7399 - loss: 0.5435
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7362 - loss: 0.5436
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7352 - loss: 0.5449
Epoch 9/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7364 - loss: 0.5485
Epoch 10/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7385 - loss: 0.5429
Epoch 11/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7316 - loss: 0.5471
Epoch 12/200
804/804 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose = 2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 1s - 3ms/step - accuracy: 0.7242 - loss: 0.5559
Loss: 0.5558626651763916, Accuracy: 0.7241982221603394


In [ ]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity.h5')